# General data

In [1]:
from bids import BIDSLayout
import pandas as pd
import numpy as np
import scipy
import json
import pyedflib
import re
import nibabel as nb
import os
import xarray as xr

In [2]:
# Load dataset
data_path = '/scratch/mcesped/Results/seegprep/hipp_run/bids/'
layout = BIDSLayout(data_path, validate=False)
layout_edf = BIDSLayout('/scratch/mcesped/Results/seegprep/hipp_run/work/', validate=False)

In [3]:
tsv_files = layout.get(extension='tsv', suffix='space')
tsv_files
# ieeg_files = layout_edf.get(extension='edf', suffix='ieeg', reconstruction='PLIreject')

[<BIDSDataFile filename='/scratch/mcesped/Results/seegprep/hipp_run/bids/sub-002/ses-002/ieeg/sub-002_ses-002_task-full_rec-regionID_run-01_regions_native_space.tsv'>,
 <BIDSDataFile filename='/scratch/mcesped/Results/seegprep/hipp_run/bids/sub-003/ses-007/ieeg/sub-003_ses-007_task-full_rec-regionID_run-02_regions_native_space.tsv'>,
 <BIDSDataFile filename='/scratch/mcesped/Results/seegprep/hipp_run/bids/sub-003/ses-008/ieeg/sub-003_ses-008_task-full_rec-regionID_run-01_regions_native_space.tsv'>,
 <BIDSDataFile filename='/scratch/mcesped/Results/seegprep/hipp_run/bids/sub-009/ses-001/ieeg/sub-009_ses-001_task-full_rec-regionID_run-01_regions_native_space.tsv'>,
 <BIDSDataFile filename='/scratch/mcesped/Results/seegprep/hipp_run/bids/sub-009/ses-002/ieeg/sub-009_ses-002_task-full_rec-regionID_run-01_regions_native_space.tsv'>,
 <BIDSDataFile filename='/scratch/mcesped/Results/seegprep/hipp_run/bids/sub-009/ses-004/ieeg/sub-009_ses-004_task-full_rec-regionID_run-01_regions_native_space

In [4]:
sub_sess_info = dict()
for tsv_file in tsv_files:
    # Get tsv file
    subj = tsv_file.get_entities()['subject']
    session = tsv_file.get_entities()['session']
    run = tsv_file.get_entities()['run']
    edf_files = layout_edf.get(extension='edf', suffix='ieeg', reconstruction='PLIreject', subject=subj, session=session, run=run, return_type='filename')
    json_file = layout.get(extension='json', subject=subj, session=session, run=run, return_type='filename')
    if len(json_file)>1:
        print('error')
    # Append to dict
    sub_sess_info[f'sub-{subj}_ses-{session}_run-{run}'] = {
        'channels_tsv': tsv_file.path,
        'edf_files' : edf_files,
        'subject': subj,
        'json': json_file[0]
    }

In [5]:
sub_sess_info[list(sub_sess_info.keys())[95]]

{'channels_tsv': '/scratch/mcesped/Results/seegprep/hipp_run/bids/sub-078/ses-008/ieeg/sub-078_ses-008_task-full_rec-regionID_run-01_regions_native_space.tsv',
 'edf_files': ['/scratch/mcesped/Results/seegprep/hipp_run/work/sub-078/ses-008/ieeg/sub-078_ses-008_task-full_rec-PLIreject_run-01_clip-01_ieeg.edf'],
 'subject': '078',
 'json': '/scratch/mcesped/Results/seegprep/hipp_run/bids/sub-078/ses-008/ieeg/sub-078_ses-008_task-full_rec-regionID_run-01_regions_native_space.json'}

# Filter based on groups

What variable do you need to create for the datarrays. This is per signal (iterable of length n = number of signals):
- subjs: subject
- chns: channel
- group: group (only 2 groups)
- subj_ses_run_chn: str identifier per signal indicating the subject, session, run and channel
- PD_coords: Proximal distal coordinate 
- AP_coords: Anterior posterior coordinate
- regions: dictionary of region for the channel

## MTS

In [6]:
label_test = 'MTS'

In [7]:
import copy
import json
# Load snsx data
snsx_path = './snsx_data_collection.csv'
snsx_df = pd.read_csv(snsx_path, sep=',')
# Load mapping snsx to clinical
mapping = pd.read_csv('mapping_snsx_clinical.tsv', sep='\t')
mapping_dict = dict(zip(mapping['ieeg_subject'].to_list(), mapping['snsx_subject'].to_list()))
# Create new data
time_data = np.array([])
# Dims (n x time)
# Coordinates: subj, chn, type (MTS_contra, MTS_ipsi), PD coord, AP coord
subj_ses_run_chn = []
subjs = []
chns = []
group = []
PD_coords = np.array([])
AP_coords = np.array([])
regions = []

for sub_sess in sub_sess_info:
    # Get bad hemispheres
    subj_clinical = sub_sess_info[sub_sess]['subject']
    subj_snsx = 'sub-' + mapping_dict[f"P{subj_clinical}"]
    # print(subj_snsx)
    generalized, bilobal, bad_R, bad_L, MTS = snsx_df[snsx_df['participant_id']==subj_snsx][['Generalized', 'Bilobal', 'Right ', 'Left', 'MRI (0=no/1=yes) MTS']].to_numpy().squeeze().astype(bool)
    # print(bad_R, bad_L)
    # Get info from channels.tsv of channels actually in the hippocampus
    loc_df = pd.read_csv(sub_sess_info[sub_sess]['channels_tsv'], sep='\t').dropna()
    # Get info per edf file
    for edf_path in sub_sess_info[sub_sess]['edf_files']:
        edf = pyedflib.EdfReader(edf_path)
        # Get chn labels
        edf_chns = edf.getSignalLabels()
        # Open json
        with open(sub_sess_info[sub_sess]['json'], 'r') as f:
          chn_info = json.load(f)
        for chn in loc_df['label'].to_numpy():
            # Check if chn is mostly in the hippocampus and type of epilepsy
            if (list(chn_info[chn].keys())[0]!='Unknown') and (not generalized) and MTS:
            # if (not generalized) and MTS:
                # Append data
                data = edf.readSignal(edf_chns.index(chn))
                time_data = np.vstack([time_data.reshape(-1,len(data)), data])
                regions.append(chn_info[chn])
                # Add coords 
                subj_ses_run_chn.append(sub_sess+f'_{chn}')
                subjs.append(subj_clinical)
                chns.append(chn)
                # Add coords AP-PD
                PD_coords = np.hstack([PD_coords, loc_df[loc_df['label']==chn]['PD'].to_numpy()])
                AP_coords = np.hstack([AP_coords, loc_df[loc_df['label']==chn]['AP'].to_numpy()])
                # Right hemi
                good_R = (loc_df[loc_df['label']==chn]['x'].values[0] > 0) and not bad_R
                # Left hemi
                good_L = (loc_df[loc_df['label']==chn]['x'].values[0] < 0) and not bad_L
                # Contra-lateral MTS TLE
                if not bilobal and (good_R or good_L):
                    group.append('contra_MTS')
                # Ipsilateral TLE
                else:
                    group.append('ipsi_MTS')   
        edf.close()

In [8]:
len(regions) #166

166

## TLE

In [6]:
label_test = 'TLE'

In [14]:
import copy
import json
# Load snsx data
snsx_path = './snsx_data_collection.csv'
snsx_df = pd.read_csv(snsx_path, sep=',')
# Load mapping snsx to clinical
mapping = pd.read_csv('mapping_snsx_clinical.tsv', sep='\t')
mapping_dict = dict(zip(mapping['ieeg_subject'].to_list(), mapping['snsx_subject'].to_list()))
# Create new data
time_data = np.array([])
# Dims (n x time)
# Coordinates: subj, chn, type (MTS_contra, MTS_ipsi), PD coord, AP coord
subj_ses_run_chn = []
subjs = []
chns = []
group = []
PD_coords = np.array([])
AP_coords = np.array([])
regions = []

for sub_sess in sub_sess_info:
    # Get bad hemispheres
    subj_clinical = sub_sess_info[sub_sess]['subject']
    subj_snsx = 'sub-' + mapping_dict[f"P{subj_clinical}"]
    # print(subj_snsx)
    generalized, bilobal, bad_R, bad_L, temporal = snsx_df[snsx_df['participant_id']==subj_snsx][['Generalized', 'Bilobal', 'Right ', 'Left', 'Temporal']].to_numpy().squeeze().astype(bool)
    # print(bad_R, bad_L)
    # Get info from channels.tsv of channels actually in the hippocampus
    loc_df = pd.read_csv(sub_sess_info[sub_sess]['channels_tsv'], sep='\t').dropna()
    # Get info per edf file
    for edf_path in sub_sess_info[sub_sess]['edf_files']:
        edf = pyedflib.EdfReader(edf_path)
        # Get chn labels
        edf_chns = edf.getSignalLabels()
        # Open json
        with open(sub_sess_info[sub_sess]['json'], 'r') as f:
          chn_info = json.load(f)
        for chn in loc_df['label'].to_numpy():
            # Check if chn is mostly in the hippocampus and type of epilepsy
            if (list(chn_info[chn].keys())[0]!='Unknown') and (not generalized) and temporal:
            # if (not generalized) and MTS:
                # Append data
                data = edf.readSignal(edf_chns.index(chn))
                time_data = np.vstack([time_data.reshape(-1,len(data)), data])
                regions.append(chn_info[chn])
                # Add coords 
                subj_ses_run_chn.append(sub_sess+f'_{chn}')
                subjs.append(subj_clinical)
                chns.append(chn)
                # Add coords AP-PD
                PD_coords = np.hstack([PD_coords, loc_df[loc_df['label']==chn]['PD'].to_numpy()])
                AP_coords = np.hstack([AP_coords, loc_df[loc_df['label']==chn]['AP'].to_numpy()])
                # Right hemi
                good_R = (loc_df[loc_df['label']==chn]['x'].values[0] > 0) and not bad_R
                # Left hemi
                good_L = (loc_df[loc_df['label']==chn]['x'].values[0] < 0) and not bad_L
                # Contra-lateral TLE
                if not bilobal and (good_R or good_L):
                    group.append('contra_TLE')
                # Ipsilateral TLE
                else:
                    group.append('ipsi_TLE')   
        edf.close()

In [15]:
len(regions) #217

217

## All: ipsi vs contra

In [6]:
label_test = 'all_epi'

In [7]:
import copy
import json
# Load snsx data
snsx_path = './snsx_data_collection.csv'
snsx_df = pd.read_csv(snsx_path, sep=',')
# Load mapping snsx to clinical
mapping = pd.read_csv('mapping_snsx_clinical.tsv', sep='\t')
mapping_dict = dict(zip(mapping['ieeg_subject'].to_list(), mapping['snsx_subject'].to_list()))
# Create new data
time_data = np.array([])
# Dims (n x time)
# Coordinates: subj, chn, type (MTS_contra, MTS_ipsi), PD coord, AP coord
subj_ses_run_chn = []
subjs = []
chns = []
group = []
PD_coords = np.array([])
AP_coords = np.array([])
regions = []

for sub_sess in sub_sess_info:
    # Get bad hemispheres
    subj_clinical = sub_sess_info[sub_sess]['subject']
    subj_snsx = 'sub-' + mapping_dict[f"P{subj_clinical}"]
    # print(subj_snsx)
    generalized, bilobal, bad_R, bad_L, temporal = snsx_df[snsx_df['participant_id']==subj_snsx][['Generalized', 'Bilobal', 'Right ', 'Left', 'Temporal']].to_numpy().squeeze().astype(bool)
    # print(bad_R, bad_L)
    # Get info from channels.tsv of channels actually in the hippocampus
    loc_df = pd.read_csv(sub_sess_info[sub_sess]['channels_tsv'], sep='\t').dropna()
    # Get info per edf file
    for edf_path in sub_sess_info[sub_sess]['edf_files']:
        edf = pyedflib.EdfReader(edf_path)
        # Get chn labels
        edf_chns = edf.getSignalLabels()
        # Open json
        with open(sub_sess_info[sub_sess]['json'], 'r') as f:
          chn_info = json.load(f)
        for chn in loc_df['label'].to_numpy():
            # Check if chn is mostly in the hippocampus and type of epilepsy. Only discarding generalized epi
            if (list(chn_info[chn].keys())[0]!='Unknown') and (not generalized):
                # Append data
                data = edf.readSignal(edf_chns.index(chn))
                time_data = np.vstack([time_data.reshape(-1,len(data)), data])
                regions.append(chn_info[chn])
                # Add coords 
                subj_ses_run_chn.append(sub_sess+f'_{chn}')
                subjs.append(subj_clinical)
                chns.append(chn)
                # Add coords AP-PD
                PD_coords = np.hstack([PD_coords, loc_df[loc_df['label']==chn]['PD'].to_numpy()])
                AP_coords = np.hstack([AP_coords, loc_df[loc_df['label']==chn]['AP'].to_numpy()])
                if temporal:
                    # Right hemi
                    good_R = (loc_df[loc_df['label']==chn]['x'].values[0] > 0) and not bad_R
                    # Left hemi
                    good_L = (loc_df[loc_df['label']==chn]['x'].values[0] < 0) and not bad_L
                    # Contra-lateral TLE
                    if not bilobal and (good_R or good_L):
                        group.append('contra_TLE')
                    # Ipsilateral TLE
                    else:
                        group.append('ipsi_TLE')
                else:
                    group.append('contra_TLE')
        edf.close()

In [8]:
len(regions) #235

235

## External dataset

In [2]:
label_test = 'extSOZ'

In [3]:
import numpy as np
import scipy.io as sio
import nibabel as nb
import mne

In [4]:
# Load matfiles
HUP_data = sio.loadmat('/home/mcesped/scratch/code/HippiEEGAtlas/code/DSP/Bernabei/Pennsieve-dataset-179-version-1/files/HUP_atlas_final.mat')
MNI_data = sio.loadmat('/scratch/mcesped/code/HippiEEGAtlas/code/DSP/MNI/MatlabFile.mat')

In [5]:
HUP_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'depth_elecs', 'mni_coords', 'patient_no', 'resected_ch', 'soz_ch', 'spike_24h', 'wake_clip'])

In [6]:
# Get channels positions and merge into an unique array
# From MNI Readme: The channel position is the midpoint between the electrode contacts that make up each bipolar channel
chn_position_HUP = HUP_data['mni_coords']
print(chn_position_HUP.shape)
chn_position_MNI = MNI_data['ChannelPosition']
print(chn_position_MNI.shape)

(3431, 3)
(1772, 3)


In [7]:
chn_positions = np.concatenate((chn_position_HUP, chn_position_MNI), axis=0)
chn_positions.shape

(5203, 3)

In [8]:
# Load dilated file to convert the position to voxels and then labels
seg_hipp = nb.load('/home/mcesped/scratch/code/HippiEEGAtlas/code/DSP/Template/mni_icbm152_nlin_sym_09a_nifti/tpl-MNI152NLin2009aSym_hemi-All_space-T1w_desc-subfields_dseg_dilated_3mm.nii.gz')

In [9]:
data_parc = seg_hipp.get_fdata()

In [10]:
# To voxels
inv_affine = np.linalg.inv(seg_hipp.affine)
# here's where the interpolation should be performed!!
vox = np.round((mne.transforms.apply_trans(inv_affine, chn_positions))).astype(int)
id_regions = data_parc[vox[:, 0], vox[:, 1], vox[:, 2]]
id_regions.shape

(5203,)

In [11]:
mask_hipp = id_regions.astype(bool)
mask_hipp.shape

(5203,)

In [12]:
len(mask_hipp[mask_hipp])

307

In [13]:
id_regions[mask_hipp]

array([7., 2., 2., 2., 4., 3., 1., 7., 5., 2., 4., 4., 6., 4., 3., 2., 4.,
       1., 7., 2., 2., 2., 2., 2., 2., 7., 2., 1., 1., 2., 7., 7., 2., 4.,
       3., 7., 2., 1., 2., 2., 6., 1., 1., 2., 2., 2., 2., 6., 2., 1., 2.,
       2., 1., 1., 2., 1., 1., 1., 2., 4., 7., 2., 2., 2., 2., 1., 4., 6.,
       2., 4., 4., 5., 4., 4., 5., 4., 7., 2., 7., 3., 1., 2., 2., 2., 1.,
       2., 4., 5., 2., 4., 5., 2., 1., 5., 2., 2., 2., 2., 1., 2., 2., 2.,
       7., 2., 5., 7., 2., 2., 4., 3., 5., 3., 2., 2., 1., 1., 2., 2., 2.,
       7., 2., 2., 4., 4., 3., 2., 2., 2., 2., 7., 5., 2., 2., 2., 2., 2.,
       2., 2., 2., 1., 3., 7., 2., 6., 2., 2., 2., 2., 2., 2., 2., 7., 2.,
       3., 2., 7., 2., 2., 7., 2., 4., 2., 2., 2., 2., 7., 2., 2., 2., 4.,
       3., 1., 2., 7., 7., 2., 1., 6., 2., 4., 3., 2., 2., 2., 2., 7., 7.,
       2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 5., 7., 7., 2., 2., 1., 2.,
       2., 2., 2., 2., 2., 2., 1., 5., 2., 2., 2., 7., 2., 3., 3., 2., 2.,
       7., 2., 2., 2., 2.

In [14]:
# Count number of subjects
subjects = np.concatenate((HUP_data['patient_no'].squeeze(), MNI_data['Patient'].squeeze()), axis=0)
subjects.shape

(5203,)

In [15]:
print(f'Lenght of subjects: {len(subjects[mask_hipp])}')
print('Unique subjects:')
len(np.unique(subjects[mask_hipp]))

Lenght of subjects: 307
Unique subjects:


55

In [16]:
# Count number of normal/abnormal channels
# 1. Load the data and classify HUP data into normal/abnormal.
# 2. All MNI channels are normal. Expand the mask based on this

In [17]:
patient_no = HUP_data['patient_no'].squeeze()
resected_ch = HUP_data['resected_ch'].squeeze().astype(bool)
soz_ch = HUP_data['soz_ch'].squeeze().astype(bool)
# From dataset description: Estimated spike rate per 24 hours (we defined irritative zone as spike_24h>24)
spike_24h = HUP_data['spike_24h'].squeeze()
spike_mask = spike_24h > 24

In [18]:
# Read metadata
# https://github.com/jbernabei/iEEG_atlas/tree/41e1ef5dc0d0ccd569aa6f2bd2c07a26f0c04b02/data
import pandas as pd
metadata = pd.read_excel('DSP/Bernabei/atlas_metadata_final.xlsx')
metadata.head()

,Patient,RID,Engel_6_mo,Engel_12_mo,Engel_24_mo,Therapy,Implant,Target,Laterality,Lesion_status,...,Age_surgery,Gender,portal_ID,clip1_awake,clip2_awake,clip1_asleep,clip2_asleep,which_file,clip3_awake,clip4_awake
0,HUP060,RID0142,3.1,3.1,3.1,Ablation,SEEG,Frontal,R,Non-Lesional,...,42,F,HUP060_phaseIV,343801,349875,387001,556201,1,423000,240179
1,HUP064,RID0054,1.4,1.4,1.4,Resection,ECoG,Frontal,L,Lesional,...,21,M,HUP64_phaseII,334801,790405,398213,475201,1,321431,440369
2,HUP065,RID0055,1.1,1.1,1.2,Resection,ECoG,Temporal,R,Lesional,...,36,M,HUP65_phaseII,507079,610604,392207,654000,1,496243,505225
3,HUP068,RID0058,1.1,1.1,NaN,Resection,ECoG,Temporal,R,Non-Lesional,...,28,F,HUP68_phaseII,513001,590371,307801,387001,1,448787,323125
4,HUP070,RID0060,1.2,1.2,1.2,Resection,ECoG,FP,L,Non-Lesional,...,33,M,HUP70_phaseII,447705,449275,383691,477872,1,452025,432781


In [19]:
# Based on the matlab code from the repo (they only grabbed the first 60 subjects)
late_outcome = np.zeros(len(metadata.index)).astype(int)
# Grab the latest available score
for idx in metadata.index:
    engel_scores = metadata.iloc[idx,2:5]
    # Drop nan values
    engel_scores = engel_scores[~engel_scores.isnull()].to_numpy()
    late_outcome[idx] = np.floor(engel_scores[-1])
late_outcome

array([3, 1, 1, 1, 1, 1, 4, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3,
       1, 1, 1, 1, 1, 3, 3, 1, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1,
       3, 1, 1, 2, 3, 2, 2, 1, 1, 3, 1, 3, 1, 2, 3, 3])

In [20]:
HUP_outcome_all = np.zeros(HUP_data['wake_clip'].shape[1]) # number of channels

for idx, score in enumerate(late_outcome):
    HUP_outcome_all[patient_no==idx+1] = score
HUP_outcome_all

array([3., 3., 3., ..., 3., 3., 3.])

In [21]:
HUP_outcome_mask = HUP_outcome_all > 1 # Based on matlab code

In [22]:
len(np.unique(patient_no[~HUP_outcome_mask]))

38

In [30]:
# HUP_mask = HUP_outcome_mask | soz_ch | spike_mask | resected_ch
# HUP masks: normal vs soz
HUP_normal_mask = ~(HUP_outcome_mask | soz_ch | spike_mask | resected_ch)
HUP_ipsi_mask = soz_ch * HUP_outcome_mask # only secting those channels that are for sure abnormal
# print(HUP_normal_mask.shape) # Booleans mask with true in abnormal channels
len(HUP_normal_mask[HUP_normal_mask]), HUP_normal_mask # number of normal channels

(532, array([False, False, False, ..., False, False, False]))

In [31]:
# Create normal mask as combination of MNI and HUP
normal_mask = np.ones(chn_positions.shape[0]).astype(bool)
print(normal_mask.shape, normal_mask)
# Repeat for soz mask
ipsi_mask = np.zeros(chn_positions.shape[0]).astype(bool)

(5203,) [ True  True  True ...  True  True  True]


In [44]:
# Introduce the HUP mask
normal_mask[0:len(HUP_normal_mask)]=HUP_normal_mask
print(normal_mask) # Booleans mask with true in normal channels
ipsi_mask[0:len(soz_ch)]=HUP_ipsi_mask
print(ipsi_mask)

[False False False ...  True  True  True]
[False False False ... False False False]


In [45]:
# Filter masks based on hippocampus mask
normal_mask_hipp = normal_mask[mask_hipp]
ipsi_mask_hipp = ipsi_mask[mask_hipp]
print(f'Normal channels in the hippocampus:{len(normal_mask_hipp[normal_mask_hipp])}')
print(f'Ipsilateral channels in the hippocampus:{len(ipsi_mask_hipp[ipsi_mask_hipp])}')
print(normal_mask_hipp.shape, ipsi_mask_hipp.shape)

Normal channels in the hippocampus:42
Ipsilateral channels in the hippocampus:28
(307,) (307,)


In [46]:
# Create combine mask to get all the signals that we need
combine_mask = normal_mask_hipp+ipsi_mask_hipp
len(combine_mask[combine_mask]), len(combine_mask)

(70, 307)

In [47]:
# Update ipsi and normal mask to have the dimensions of only the relevant data indicated by the combine mask
normal_mask_hipp = normal_mask_hipp[combine_mask]
ipsi_mask_hipp = ipsi_mask_hipp[combine_mask]
print(normal_mask_hipp.shape, ipsi_mask_hipp.shape)

(70,) (70,)


In [48]:
# View some of the results
# Load data
HUP_signals = HUP_data['wake_clip']
print(HUP_signals.shape)
MNI_signals = MNI_data['Data_W']
print(MNI_signals.shape)

(12000, 3431)
(13600, 1772)


In [49]:
# Cut MNI signals to concatenate with HUP
merged_signals = np.concatenate((HUP_signals, MNI_signals[0:HUP_signals.shape[0]]), axis=1)
merged_signals.shape

(12000, 5203)

In [50]:
# Get signals in the hippocampus
hipp_signals = merged_signals[:,mask_hipp]
hipp_signals.shape

(12000, 307)

In [51]:
normal_mask_hipp.shape

(70,)

In [52]:
# Define data for dataset
time_data = hipp_signals.T[combine_mask]
chns =  ['{:03}'.format(number) for number in np.arange(time_data.shape[0])]
subjs = subjects[mask_hipp][combine_mask]
# Group
group = np.zeros(time_data.shape[0]).astype(str)
group[:] = 'ipsi'
group[normal_mask_hipp] = 'normal'
# Identifier
subj_chn = [f'subj-{subj}_chn-{chn}' for subj, chn in list(zip(subjs, chns))]

In [53]:
# Convert to xarray dataset
ds_time = xr.Dataset(
    {'time_domain' : (['n', 'time'], time_data)},
    coords={
        "subj": (["n"], subjs),
        "chn": (["n"], chns),
        "group": (["n"], group),
        "subj_chn": (["n"], subj_chn),
        "time": np.arange(time_data.shape[-1])/200,
        'n': np.arange(time_data.shape[0])
    },
)
ds_time

<xarray.Dataset>
Dimensions:      (n: 70, time: 12000)
Coordinates:
    subj         (n) uint8 13 19 19 22 22 27 29 29 ... 74 74 74 77 77 77 77 90
    chn          (n) <U3 '000' '001' '002' '003' ... '066' '067' '068' '069'
    group        (n) <U32 'normal' 'normal' 'normal' ... 'normal' 'normal'
    subj_chn     (n) <U15 'subj-13_chn-000' ... 'subj-90_chn-069'
  * time         (time) float64 0.0 0.005 0.01 0.015 ... 59.98 59.98 59.99 59.99
  * n            (n) int64 0 1 2 3 4 5 6 7 8 9 ... 60 61 62 63 64 65 66 67 68 69
Data variables:
    time_domain  (n, time) float64 17.23 19.3 20.63 ... -4.576 -6.635 -7.119

In [54]:
ds_time.where(ds_time['group']=='normal', drop=True)

<xarray.Dataset>
Dimensions:      (n: 42, time: 12000)
Coordinates:
    subj         (n) uint8 13 19 19 27 30 34 34 35 ... 74 74 74 77 77 77 77 90
    chn          (n) <U3 '000' '001' '002' '005' ... '066' '067' '068' '069'
    group        (n) <U32 'normal' 'normal' 'normal' ... 'normal' 'normal'
    subj_chn     (n) <U15 'subj-13_chn-000' ... 'subj-90_chn-069'
  * time         (time) float64 0.0 0.005 0.01 0.015 ... 59.98 59.98 59.99 59.99
  * n            (n) int64 0 1 2 5 8 14 15 16 19 ... 61 62 63 64 65 66 67 68 69
Data variables:
    time_domain  (n, time) float64 17.23 19.3 20.63 ... -4.576 -6.635 -7.119

In [55]:
os.makedirs(f'./Data/{label_test}/', exist_ok=True)

# Save regions data

In [9]:
os.makedirs(f'./Data/{label_test}/', exist_ok=True)
with open(f'./Data/{label_test}/regions_{label_test}.json', 'w') as f:
    json.dump(regions, f)

In [10]:
# Convert to xarray dataset
ds_time = xr.Dataset(
    {'time_domain' : (['n', 'time'], time_data)},
    coords={
        "subj": (["n"], subjs),
        "chn": (["n"], chns),
        "group": (["n"], group),
        "subj_ses_run_chn": (["n"], subj_ses_run_chn),
        "PD": (["n"], PD_coords),
        "AP": (["n"], AP_coords),
        "time": np.arange(time_data.shape[-1])/200,
        'n': np.arange(time_data.shape[0])
    },
)

# Convert to frequency domain

In [11]:
# Get data in frequency domain
import analysis
ds_freq = analysis.psd_xarray(ds_time, 200)

(235, 201)


In [12]:
ds_freq['psd']

<xarray.DataArray 'psd' (n: 235, frequency: 159)>
array([[5.94570958e-01, 5.19615297e-01, 3.98595728e-01, ...,
        2.24994034e-04, 1.98859915e-04, 1.82777833e-04],
       [7.02003924e-01, 5.44063056e-01, 3.23124599e-01, ...,
        5.19682031e-04, 5.53606430e-04, 5.67792656e-04],
       [5.83397882e-01, 5.31296592e-01, 3.30196479e-01, ...,
        9.64287178e-04, 9.45979014e-04, 9.86152411e-04],
       ...,
       [6.49960008e-01, 5.17339487e-01, 3.32054415e-01, ...,
        3.20705548e-04, 2.79240126e-04, 2.95289291e-04],
       [5.48033271e-01, 5.46305503e-01, 3.65660798e-01, ...,
        4.40677674e-04, 4.01821177e-04, 4.31011236e-04],
       [5.76290716e-01, 5.49682622e-01, 3.65805878e-01, ...,
        3.23369996e-04, 3.15551027e-04, 3.16208554e-04]])
Coordinates:
    subj              (n) <U3 '003' '003' '003' '003' ... '095' '095' '095'
    chn               (n) <U9 'LAHc1-2' 'RAHc1-2' ... 'LAHc1-2' 'LAHc2-3'
    group             (n) <U10 'contra_TLE' 'ipsi_TLE' ... 'ipsi_TLE' 'ipsi_TLE'
    subj_ses_run_chn  (n) <U32 'sub-003_ses-007_run-02_LAHc1-2' ... 'sub-095_...
    PD                (n) float64 0.9948 0.9887 0.995 ... 0.9988 0.872 0.9988
    AP                (n) float64 0.3091 0.2321 0.2765 ... 0.301 0.2531 0.301
  * n                 (n) int64 0 1 2 3 4 5 6 7 ... 228 229 230 231 232 233 234
  * frequency         (frequency) float64 0.5 1.0 1.5 2.0 ... 78.5 79.0 79.5

In [13]:
# Reduced freq
ds_freq_reduced = analysis.average_per_channel(ds_freq['psd'])
ds_freq_reduced

<xarray.Dataset>
Dimensions:    (n: 34, frequency: 159)
Coordinates:
    subj       (n) <U3 '003' '003' '003' '003' '003' ... '083' '088' '095' '095'
    chn        (n) <U9 'LAHc1-2' 'RAHc1-2' 'RAHc2-3' ... 'LAHc1-2' 'LAHc2-3'
    group      (n) <U10 'contra_TLE' 'ipsi_TLE' ... 'ipsi_TLE' 'ipsi_TLE'
    subj_chn   (n) <U17 'sub-003_LAHc1-2' ... 'sub-095_LAHc2-3'
    PD         (n) float64 0.9948 0.9887 0.995 0.3603 ... 0.2447 0.872 0.9988
    AP         (n) float64 0.3091 0.2321 0.2765 0.6129 ... 0.2689 0.2531 0.301
  * frequency  (frequency) float64 0.5 1.0 1.5 2.0 2.5 ... 78.0 78.5 79.0 79.5
  * n          (n) int64 0 1 2 3 4 5 6 7 8 9 ... 24 25 26 27 28 29 30 31 32 33
Data variables:
    psd        (n, frequency) float64 0.5852 0.5136 ... 0.0003551 0.0003577

# Bandpower

In [14]:
ds_bp = analysis.compute_bandpower(ds_freq, 'group')
ds_bp

<xarray.Dataset>
Dimensions:           (n: 235, time: 48000, frequency: 159, f_bands: 5)
Coordinates:
    subj              (n) <U3 '003' '003' '003' '003' ... '095' '095' '095'
    chn               (n) <U9 'LAHc1-2' 'RAHc1-2' ... 'LAHc1-2' 'LAHc2-3'
    group             (n) <U10 'contra_TLE' 'ipsi_TLE' ... 'ipsi_TLE' 'ipsi_TLE'
    subj_ses_run_chn  (n) <U32 'sub-003_ses-007_run-02_LAHc1-2' ... 'sub-095_...
    PD                (n) float64 0.9948 0.9887 0.995 ... 0.9988 0.872 0.9988
    AP                (n) float64 0.3091 0.2321 0.2765 ... 0.301 0.2531 0.301
  * time              (time) float64 0.0 0.005 0.01 0.015 ... 240.0 240.0 240.0
  * n                 (n) int64 0 1 2 3 4 5 6 7 ... 228 229 230 231 232 233 234
  * frequency         (frequency) float64 0.5 1.0 1.5 2.0 ... 78.5 79.0 79.5
  * f_bands           (f_bands) <U5 'delta' 'theta' 'alpha' 'beta' 'gamma'
Data variables:
    time_domain       (n, time) float64 -0.1329 -3.589 -70.32 ... 59.68 -0.1329
    psd               (n, frequency) float64 0.5946 0.5196 ... 0.0003162
    bandpow           (n, f_bands) float64 0.765 0.1212 ... 0.0522 0.03408

In [15]:
ds_bp.to_netcdf(f"./Data/{label_test}/ds_{label_test}_full.nc")

In [16]:
# Reduced bandpower
ds_bp_reduced = analysis.compute_bandpower(ds_freq_reduced, 'group')
ds_bp_reduced

<xarray.Dataset>
Dimensions:    (n: 34, frequency: 159, f_bands: 5)
Coordinates:
    subj       (n) <U3 '003' '003' '003' '003' '003' ... '083' '088' '095' '095'
    chn        (n) <U9 'LAHc1-2' 'RAHc1-2' 'RAHc2-3' ... 'LAHc1-2' 'LAHc2-3'
    group      (n) <U10 'contra_TLE' 'ipsi_TLE' ... 'ipsi_TLE' 'ipsi_TLE'
    subj_chn   (n) <U17 'sub-003_LAHc1-2' ... 'sub-095_LAHc2-3'
    PD         (n) float64 0.9948 0.9887 0.995 0.3603 ... 0.2447 0.872 0.9988
    AP         (n) float64 0.3091 0.2321 0.2765 0.6129 ... 0.2689 0.2531 0.301
  * frequency  (frequency) float64 0.5 1.0 1.5 2.0 2.5 ... 78.0 78.5 79.0 79.5
  * n          (n) int64 0 1 2 3 4 5 6 7 8 9 ... 24 25 26 27 28 29 30 31 32 33
  * f_bands    (f_bands) <U5 'delta' 'theta' 'alpha' 'beta' 'gamma'
Data variables:
    psd        (n, frequency) float64 0.5852 0.5136 ... 0.0003551 0.0003577
    bandpow    (n, f_bands) float64 0.7598 0.123 0.04534 ... 0.06019 0.03586

In [17]:
ds_bp_reduced.to_netcdf(f"./Data/{label_test}/ds_{label_test}_freq_reduced.nc")